Συμπλήρωση ελλιπών τιμών με τρείς διαφορετικές μεοδολογίες, Knn, Iterative imputation, multiple imputation miceforest.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')
pd.options.display.float_format = '{:,.3f}'.format

In [2]:
from sklearn.preprocessing import MinMaxScaler

In [3]:
from numpy import isnan

In [4]:
#data WITHOUT near-zero value columns
data = pd.read_csv('data2_droplists.csv')

In [5]:
data.shape

(1838, 668)

In [6]:
#make sure dataset has only numbers
data = data.apply(pd.to_numeric)

In [8]:
print("Number of null values in dataset: ",data.isnull().values.sum())
print("Percentage: ",(data.isnull().sum().sum()/data.size)*100) #ποσοστο

Number of null values in dataset:  97121
Percentage:  7.910267604073681


In [9]:
data['G1'].value_counts()

0.000    1514
1.000     240
2.000      59
3.000      14
4.000      11
Name: G1, dtype: int64

In [10]:
#data['G1']=data['G1'].fillna(0,inplace=True)
data['G1'].isnull().values.sum()

0

### scale, y = G1 

In [11]:
#load data and split to dependent and independent variables
X = data.loc[:, data.columns != 'G1'].values #select all but the desired one for y
y = data['G1'].values

#X and y are numpy ndarray type variables
print(X.shape,y.shape)
X = np.array(X)
y = np.array(y)

(1838, 667) (1838,)


In [12]:
# MinMaxScaler() normalizes, range on the training set, default=(0, 1)
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

#### kNN

In [13]:
from sklearn.impute import KNNImputer
# define imputer
knn_imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

In [14]:
# fit on the dataset
knn_imputer.fit(X)

KNNImputer()

In [15]:
# transform the dataset
X_knn= knn_imputer.transform(X)

In [16]:
print('Missing: %d' % sum(isnan(X_knn).flatten()))

Missing: 0


#### Iterative Imputer

In [17]:
# define imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

iter_imputer =IterativeImputer(max_iter=5,n_nearest_features=200,initial_strategy='most_frequent',imputation_order='ascending', verbose=1)

In [18]:
# fit on the dataset
iter_imputer.fit(X)

[IterativeImputer] Completing matrix with shape (1838, 667)
[IterativeImputer] Change: 40.57114643932917, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Change: 32.23506094078855, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Change: 23.905498432135644, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Change: 32.687954433487185, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Change: 32.25789698240602, scaled tolerance: 0.0010000000000000002 


C:\Users\redi\Anaconda3\envs\tf\lib\site-packages\sklearn\impute\_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


IterativeImputer(initial_strategy='most_frequent', max_iter=5,
                 n_nearest_features=200, verbose=1)

In [19]:
# transform the dataset
X_iter= iter_imputer.transform(X)

[IterativeImputer] Completing matrix with shape (1838, 667)


In [20]:
print('Missing: %d' % sum(isnan(X_iter).flatten()))

Missing: 0


In [21]:
from numpy import asarray
from numpy import save

#save different X
save('X_G1knn.npy', X_knn)
save('X_G1iter.npy', X_iter)
#save y for G1
save('y_G1.npy', y)

#### Multiple impution - miceforest

In [214]:
# define imputer
import miceforest as mf

# Create kernel
kds = mf.ImputationKernel(X, datasets=4, mean_match_candidates=5, data_subset=5, save_all_iterations=False,save_models=0, copy_data=False)


In [215]:
kds.mice(10)

In [216]:
print(kds)

              Class: ImputationKernel
           Datasets: 4
         Iterations: 10
  Imputed Variables: 626
save_all_iterations: False


In [217]:
X_mice = kds.complete_data(dataset=0, inplace=False)

In [218]:
print('Missing: %d' % sum(isnan(X_mice).flatten()))

Missing: 0


In [ ]:
save('X_G1mice10.npy', X_mice)